In [1]:
def read_file(FILE) :
    count = 0
    with open(FILE) as f :
        f.readline()
        Hs = dict()
        Vs = dict()
        for l in f :
            l = l[:-1].split(" ")
            if l[0] == "H" : 
                Hs[count] = l[2:]
            else :
                Vs[count] = l[2:]
            count += 1
        return Hs,Vs
    print("error")

In [2]:
def index_by_tags(H,V) :
    tags_H = dict()    
    tags_V = dict()
    for h in H :
        for t in H[h]:
            if t not in tags_H :
                tags_H[t] = []
                tags_V[t] = []
            tags_H[t].append(h)
    for v in V :
        for t in V[v]:
            if t not in tags_V :
                tags_V[t] = []
                tags_H[t] = []
            tags_V[t].append(v)
    return tags_H,tags_V

In [3]:
def merge(l1, l2):
    l = l1[:]
    for elt in l2:
        if(not elt in l):
            l.append(elt)
    return l

def score(tags1, tags2):
    if(type(tags1) is tuple):
        tags1 = merge(tags1[0], tags1[1])
    if(type(tags2) is tuple):
        tags2 = merge(tags2[0], tags2[1])
    
    in1 = 0
    in2 = 0
    inter = 0
    for t in tags1:
        if(t in tags2):
            inter += 1
    in1 = len(tags1) - inter
    in2 = len(tags2) - inter
    return min(min(in1, in2), inter)

In [4]:
def get_intersection(t1,t2) :
    return [t for t in t1 if t in t2]

In [5]:
def get_best_next_V(V, tV, last, nb_tests):
    max_score = -1
    best_id1 = None
    it = iter(V)
    
    to_be_tested = []
    for t in last :
        if(len(to_be_tested) >= nb_tests):
            break
        for v in tV[t] :
            if v in V :
                to_be_tested.append(v)
    for v in to_be_tested[:nb_tests]:
        s = score(last,V[v])
        if s > max_score:
            best_id1 = v
            max_score = s
    if best_id1 == None :
        best_id1 = next(it)

    max_score = -1
    tags1 = V[best_id1]
    best_id2 = None
    
    to_be_tested = []
    for t in last :
        if(len(to_be_tested) >= nb_tests):
            break
        if(not t in tags1):
            for v in tV[t] :
                if v in V :
                    to_be_tested.append(v)
    for v in to_be_tested[:nb_tests]:
        if(v == best_id1):
            continue
        s = score(last, (tags1,V[v]))
        if s > max_score:
            best_id2 = v
            max_score = s
    if best_id2 == None :
        best_id2 = next(it)
        if(best_id2 == best_id1):
            best_id2 = next(it)
        
    return (best_id1, best_id2, score(last, (V[best_id1], V[best_id2])))

In [6]:
def get_best_next_H(H,tH,last,nb_tests) :
    max_score = -1
    best_id = None
    to_bo_tested = []
    for t in last :
        for h in tH[t] :
            if h in H :
                to_bo_tested.append(h)
    for h in to_bo_tested[:nb_tests]:
        s = score(last,H[h])
        if s > max_score:
            best_id = h
            max_score = s
    if best_id == None :
        best_id = next(iter(H))
        max_score = 0
    return best_id,max_score
        

def greedy_algo_H(FILE, nb_tests=100) :
    total_score = 0
    H,V = read_file(FILE)
    tH,Tv = index_by_tags(H,V)
    res = []
    last_slide = next(iter(H))
    res.append(last_slide)
    last = H[last_slide]
    del H[last_slide]
    count_zeros = 0
    while(H) :
        last_slide,s = get_best_next_H(H,tH,last,nb_tests)
        res.append(last_slide)
        last = H[last_slide]
        del H[last_slide]
        if not s :
            count_zeros += 1
        total_score += s
    print("Score : %d" % total_score)
    print("Zeros : %d" % count_zeros)
    return res

In [7]:
def greedy_algo(FILE, nb_tests=100) :
    total_score = 0
    H,V = read_file(FILE)
    tH,tV = index_by_tags(H,V)
    res = []
    if(H):
        last_slide = next(iter(H))
        last = H[last_slide]
        del H[last_slide]
    else:
        it = iter(V)
        s1 = next(it)
        s2 = next(it)
        last_slide = (s1, s2)
        last = merge(V[s1], V[s2])
        del V[s1]
        del V[s2]
    
    res.append(last_slide)
    count = 0
    while(len(H) > 0 or len(V) >= 2) :
        sH, sV = -1, -1
        slide, slide1, slide2 = None, None, None
        if(len(H) > 0):
            slide,sH = get_best_next_H(H,tH,last,nb_tests)
        if(len(V) >= 2):
            slide1, slide2, sV = get_best_next_V(V, tV, last, nb_tests)
            
        if(sH > sV):
            last_slide = slide
            last = H[slide]
            del H[last_slide]
            total_score += sH
        else:
            last_slide = (slide1, slide2)
            last = merge(V[slide1], V[slide2])
            del V[slide1]
            del V[slide2]
            total_score += sV
        res.append(last_slide)
        count += 1
        if(count % 1000 == 0):
            print(count)
    print("Score : %d" % total_score)
    return res

In [ ]:
FILE = "d_pet_pictures.txt"
res = greedy_algo(FILE,10)

1000
2000
3000


In [146]:
H,V = read_file(FILE)
r = index_by_tags(H,V)

In [62]:
def write_submission(res, name) :
    with open(name,"w") as file :
        file.write("%d\n" % len(res))
        for r in res :
            if type(r) is tuple :
                file.write("%d %d\n" % r)
            else :
                file.write("%d\n" % r)

In [116]:
write_submission(res, "b.txt")